In [8]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr

In [39]:
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [40]:
system_message = 'You are a helpful assistant'

In [46]:
def message_llama(prompt):
    message= [
        {"role": "system", "content": system_message},
        {"role": "user", "content":prompt}
    ]
    completion = openai.chat.completions.create(
        model="qwen3", messages=message
    )
    return completion.choices[0].message.content

In [45]:
message_llama("what version of large language model are you?")

"<think>\nOkay, the user is asking what version of large language model I am. I need to make sure I answer accurately. First, I recall that I am Qwen, a large language model developed by Alibaba Cloud. My version is Qwen, which is part of the Qwen series. The user might be interested in knowing about my specific version or if there are different versions like Qwen, Qwen2, etc.\n\nI should confirm if they are referring to a specific version or just the general model. Also, since the user is using English, they might be from an English-speaking region or just prefer English. I should respond in English as usual. Let me check the available versions. The Qwen series includes multiple versions, such as Qwen, Qwen2, Qwen3, and others. But if they don't specify, I can mention the main version and offer to provide more details if needed. I should keep the answer concise but informative.\n</think>\n\nI am Qwen, a large language model developed by Alibaba Cloud. There are multiple versions in th

In [48]:
def shout(text):
    return text.upper()

In [49]:
shout("hello")

'HELLO'

In [50]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", allow_flagging="never").launch(share=True)

Running on local URL:  http://127.0.0.1:7866
Running on public URL: https://f981d18dcb0e849bc2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [51]:
gr.Interface(fn=message_llama, inputs='textbox', outputs='textbox', allow_flagging='never').launch()

Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


In [52]:
system_message = 'you are a helpful assistant that responds in markdown'

view = gr.Interface(fn=message_llama,
                    inputs=[gr.Textbox(label="Your Message")],
                    outputs=[gr.Markdown(label="Response")],
                    allow_flagging='never'

                    )
view.launch()

Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


In [53]:
def stream_ollama (prompt):
    message=[
        {"role":"system", "content": system_message},
        {"role": "user", "content":prompt}
    ]
    stream = openai.chat.completions.create(
        model="qwen3", messages=message,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield  result



In [55]:
view = gr.Interface(
    fn=stream_ollama,
                   inputs=[gr.Textbox(label="Your Message:")],
                   outputs=[gr.Markdown(label="Response:")],
                   allow_flagging="never")
view.launch()

Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


In [56]:
def stream_ollama32 (text):
    message=[
        {"role":"system", "content": system_message},
        {"role": "user", "content":text}
    ]
    stream = openai.chat.completions.create(
        model="llama3.2", messages=message,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield  result


In [57]:
def stream_model(prompt,model):
    if model=="qwen3":
        result = stream_ollama(prompt)
    elif model=="llama3.2":
       result = stream_ollama32(prompt)
    else:
       raise ValueError("Unknown model")
    for chunk in result:
            yield chunk


In [58]:
view = gr.Interface(fn=stream_model,
                    inputs=[gr.Textbox(label="Your Message:"),
                    gr.Dropdown(["qwen3","llama3.2"], label="select Model")],
                    outputs=[gr.Markdown(label="Response:")], allow_flagging="never"
                    )
view.launch()

Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.
